Geramos dois times com 15 jogadores cada e as seguintes colunas:

- player_id;
- rating;
- k_value;
- team;
- age;
- games_played.

As informações de idade e jogos jogados são gerados aleatoriamente entre um intervalo de valores. *k* é inicializado como 0 mas será atualizado na primeira partida. Depois, é adicionado 3 colunas na tabela:

- minutes_played;
- goals_for;
- goals_againt.

Que são informações que representam exclusivamente a última partida jogada. As informações são referentes ao placar do jogo quando o jogador esteve em campo.


In [1]:
import pandas as pd
import numpy as np

def create_team_data(player_ids, team_name, rating):
    n_players = len(player_ids)
    

    team_data = pd.DataFrame({
        'player_id': player_ids,
        'rating': rating,
        'k_value': 0,
        'team': team_name,
        'age': np.random.randint(16, 35, n_players),
        'games_played': np.random.randint(22, 40, n_players),
    })
    
    return team_data

team_a = create_team_data(range(1, 16), 'A', 2500)
team_b = create_team_data(range(17, 32), 'B', 1500)

team_a

,player_id,rating,k_value,team,age,games_played
0,1,2500,0,A,17,37
1,2,2500,0,A,28,38
2,3,2500,0,A,19,23
3,4,2500,0,A,21,22
4,5,2500,0,A,21,39
5,6,2500,0,A,34,39
6,7,2500,0,A,20,37
7,8,2500,0,A,21,24
8,9,2500,0,A,34,31
9,10,2500,0,A,20,23


In [2]:
def update_k_q_values(player):
    if (player['games_played'] < 30 or player['age'] < 18) and player['rating'] < 2300:
        player['k_value'] = 40
    elif player['rating'] < 2400:
        player['k_value'] = 20
    elif player['games_played'] >= 30 and player['rating'] >= 2400:
        player['k_value'] = 10
    else:
        player['k_value'] = 10  

    player['q_value'] = 0.5

    return player

def calculate_team_rating(team):
    total_minutes = team['minutes_played'].sum()
    return np.sum(team['rating'] * team['minutes_played']) / total_minutes

def expected_score(ra, rb):
    return 1 / (1 + 10 ** ((rb - ra) / 400))

In [3]:
def calculate_individual_changes(team_1, team_2, w = 1, q = 0.5):
    opponent_team = team_2

    Rb = calculate_team_rating(opponent_team)

    Mmax = 90
    for index, player in team_1.iterrows():
        Ea = expected_score(player['rating'], Rb)

        Da = player['goals_for'] - player['goals_against']

        Sa = 1 if Da > 0 else 0.5 if Da == 0 else 0

        if Da!=0:
            Ca = w * (Sa - Ea) * np.cbrt(abs(Da))
        else:
            Ca = w * (Sa - Ea) * player['minutes_played']/Mmax
        
        player = update_k_q_values(player)
        team_1.loc[index, 'k_value'] = player['k_value']
        new_rating = player['rating'] + player['k_value'] * ((q * Ca) + ((1 - q) * Ca * (player['minutes_played'] / 90)))
        team_1.loc[index, 'rating'] = new_rating
        
        team_1.loc[index, 'games_played'] += 1

    return team_1

In [4]:
def simulate_match(team_a, team_b):

    for team in [team_a, team_b]:

        n_players = team_a.shape[0]
        minutes_played = np.zeros(n_players, dtype=int)

        full_time_players = np.random.choice(range(11), size=int(0.75 * 11), replace=False) # select 11 random players to play the full 90 minutes
        minutes_played[full_time_players] = 90
        
        substitute_needed = [x for x in range(11) if x not in full_time_players] # find the players who need to be substituted

        for player in substitute_needed:
            minutes_played[player] = np.random.randint(50, 90)

        for player in substitute_needed:
            for sub in range(11, 14):
                if minutes_played[sub] == 0:
                    minutes_played[sub] = 90 - minutes_played[player]
                    break

        team['minutes_played'] = minutes_played

    for team in [team_a, team_b]:
        if team is team_a:
            random_goal_for = np.random.randint(0, 4)
            random_goal_against = np.random.randint(0, 4)
        else:
            random_goal_for = random_goal_against
            random_goal_against = random_goal_for

        for index, row in team.iterrows():
            if row['minutes_played'] == 90:
                team.at[index, 'goals_for'] = random_goal_for
                team.at[index, 'goals_against'] = random_goal_against
            else:
                team.at[index, 'goals_for'] = np.random.randint(0, random_goal_for + 1)
                team.at[index, 'goals_against'] = np.random.randint(0, random_goal_against + 1)


    home_advantage = 100
    team_a['rating'] += home_advantage

    team_a = calculate_individual_changes(team_a, team_b)
    team_b = calculate_individual_changes(team_b, team_a)

    team_a['rating'] -= home_advantage  

    return team_a, team_b

def simulate_n_matches(n, team_a, team_b):
    for i in range(n):
        team_a, team_b = simulate_match(team_a, team_b)
    
    return team_a, team_b

In [5]:
team_a = create_team_data(range(1, 16), 'A', 2500)
team_b = create_team_data(range(17, 32), 'B', 1500)


team_a

,player_id,rating,k_value,team,age,games_played
0,1,2500,0,A,33,38
1,2,2500,0,A,30,28
2,3,2500,0,A,21,22
3,4,2500,0,A,16,33
4,5,2500,0,A,30,37
5,6,2500,0,A,27,33
6,7,2500,0,A,32,35
7,8,2500,0,A,16,36
8,9,2500,0,A,21,28
9,10,2500,0,A,29,35


In [6]:
team_b

,player_id,rating,k_value,team,age,games_played
0,17,1500,0,B,28,33
1,18,1500,0,B,17,26
2,19,1500,0,B,19,33
3,20,1500,0,B,30,35
4,21,1500,0,B,27,22
5,22,1500,0,B,29,39
6,23,1500,0,B,29,38
7,24,1500,0,B,27,32
8,25,1500,0,B,30,22
9,26,1500,0,B,18,28


In [7]:
n = 5
team_a, team_b = simulate_n_matches(n, team_a, team_b)

In [8]:
team_a

,player_id,rating,k_value,team,age,games_played,minutes_played,goals_for,goals_against
0,1,2467.522103,10,A,33,43,90,0.0,1.0
1,2,2466.749016,10,A,30,33,90,0.0,1.0
2,3,2467.531272,10,A,21,27,90,0.0,1.0
3,4,2463.775577,10,A,16,38,90,0.0,1.0
4,5,2473.183762,10,A,30,42,84,0.0,0.0
5,6,2467.531272,10,A,27,38,90,0.0,1.0
6,7,2481.710373,10,A,32,40,73,0.0,0.0
7,8,2463.478056,10,A,16,41,90,0.0,1.0
8,9,2467.531272,10,A,21,33,90,0.0,1.0
9,10,2467.807833,10,A,29,40,85,0.0,1.0


In [9]:
team_b

,player_id,rating,k_value,team,age,games_played,minutes_played,goals_for,goals_against
0,17,1546.996645,20,B,28,38,90,1.0,1.0
1,18,1579.508694,40,B,17,31,87,0.0,1.0
2,19,1545.230717,20,B,19,38,90,1.0,1.0
3,20,1549.350142,20,B,30,40,50,0.0,1.0
4,21,1597.247137,40,B,27,27,90,1.0,1.0
5,22,1545.752638,20,B,29,44,90,1.0,1.0
6,23,1549.453861,20,B,29,43,90,1.0,1.0
7,24,1539.785871,20,B,27,37,81,0.0,1.0
8,25,1599.506460,40,B,30,27,90,1.0,1.0
9,26,1550.589575,20,B,18,33,90,1.0,1.0
